## 构建RAG应用

In [1]:
import os
from dotenv import load_dotenv, find_dotenv 

_ = load_dotenv(find_dotenv())
openai_api_key = os.environ['OPENAI_API_KEY']

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0.0)
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x7f6e416381c0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7f6e2b46cfa0>, root_client=<openai.OpenAI object at 0x7f6e2b76ea70>, root_async_client=<openai.AsyncOpenAI object at 0x7f6e4163b730>, temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [3]:
output = llm.invoke("What is the capital of France?")
output

AIMessage(content='The capital of France is Paris.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 14, 'total_tokens': 21, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a893ca92-5ecf-42a8-a4b3-e34f0bec5ff1-0', usage_metadata={'input_tokens': 14, 'output_tokens': 7, 'total_tokens': 21})

In [4]:
# 这里我们要求模型对给定文本进行中文翻译
prompt = """请你将由三个反引号分割的文本翻译成英文！\
text: ```{text}```
"""

In [5]:
text = "我带着比身体重的行李，\
游入尼罗河底，\
经过几道闪电 看到一堆光圈，\
不确定是不是这里。\
"
prompt.format(text=text)

'请你将由三个反引号分割的文本翻译成英文！text: ```我带着比身体重的行李，游入尼罗河底，经过几道闪电 看到一堆光圈，不确定是不是这里。```\n'

In [6]:
from langchain_core.prompts import ChatPromptTemplate

template = "你是一个翻译助手，可以帮助我将 {input_language} 翻译成 {output_language}."
human_template = "{text}"

chat_prompt = ChatPromptTemplate([("system", template), ("human", human_template)])

text = "我带着比身体重的行李，\
游入尼罗河底，\
经过几道闪电 看到一堆光圈，\
不确定是不是这里。\
"
messages = chat_prompt.invoke({"input_language": "中文", "output_language": "英文", "text": text})
messages

ChatPromptValue(messages=[SystemMessage(content='你是一个翻译助手，可以帮助我将 中文 翻译成 英文.', additional_kwargs={}, response_metadata={}), HumanMessage(content='我带着比身体重的行李，游入尼罗河底，经过几道闪电 看到一堆光圈，不确定是不是这里。', additional_kwargs={}, response_metadata={})])

In [7]:
output = llm.invoke(messages)
output

AIMessage(content='I carried luggage heavier than my body and dived into the bottom of the Nile River. After passing through several flashes of lightning, I saw a pile of halos, not sure if this is the place.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 95, 'total_tokens': 137, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b17de4a1-685a-4028-909e-fdf226023916-0', usage_metadata={'input_tokens': 95, 'output_tokens': 42, 'total_tokens': 137})

In [8]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
output_parser.invoke(output)

'I carried luggage heavier than my body and dived into the bottom of the Nile River. After passing through several flashes of lightning, I saw a pile of halos, not sure if this is the place.'

#### 组合成一个链条

In [9]:
chain = chat_prompt | llm | output_parser # LCEL模板语法

In [10]:
text = 'I carried luggage heavier than my body and dived into the bottom of the Nile River. After passing through several flashes of lightning, I saw a pile of halos, not sure if this is the place.'
chain.invoke({"input_language": "英文", "output_language": "中文","text": text})

'我扛着比我的身体还重的行李，潜入了尼罗河的底部。穿过几道闪电后，我看到了一堆光环，不确定这是否就是目的地。'

### 构建检索问答链

#### 加载向量数据库

In [11]:
import sys

sys.path.append("./llm-universe/notebook/C4 构建 RAG 应用")

from langchain.vectorstores import Chroma

In [12]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

openai_api_key = os.environ['OPENAI_API_KEY']

In [13]:
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

persist_directory = './llm-universe/data_base/vector_db/chroma'

vector_db = Chroma(
  persist_directory=persist_directory,
  embedding_function=embeddings
)

/tmp/ipykernel_18932/3977479060.py:3: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  embeddings = OpenAIEmbeddings()
/tmp/ipykernel_18932/3977479060.py:7: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  vector_db = Chroma(


In [14]:
print(f"向量库中存储的数量：{vector_db._collection.count()}")

向量库中存储的数量：0


In [15]:
question = "什么是 prompt engineering？"
retriever = vector_db.as_retriever(search_keywords={'k': 3})
docs = retriever.invoke(question)
print(f"检索到的内容数：{len(docs)}")

检索到的内容数：0


In [16]:
for i, doc in enumerate(docs):
    print(f"检索到的第{i}个内容: \n {doc.page_content}", end="\n-----------------------------------------------------\n")

#### 创建检索链

In [17]:
from langchain_core.runnables import RunnableLambda

def combine_docs(docs):
  return "\n\n".join(doc.page_content for doc in docs)

combiner = RunnableLambda(combine_docs)
retrieval_chain = retriever | combiner
retrieval_chain.invoke("南瓜书是什么？")

''

#### 创建 LLM

In [18]:
import os

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

In [19]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o', temperature=0.0)

llm.invoke('请你介绍一下你自己').content

'当然可以！我是一个由OpenAI开发的人工智能助手，名叫ChatGPT。我擅长处理自然语言，可以帮助回答问题、提供信息、协助解决问题以及进行各种对话。我没有个人意识或情感，只是一个基于大量数据训练的语言模型，旨在为用户提供有用和准确的回答。如果你有任何问题或需要帮助，随时可以问我！'

In [21]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

template = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。最多使用三句话。尽量使答案简明扼要。请你在回答的最后说“谢谢你的提问！”。
{context}
问题: {input}
"""

prompt = PromptTemplate(template=template)

qa_chain = (RunnableParallel({'context': retrieval_chain, 'input': RunnablePassthrough()}) | prompt | llm | StrOutputParser())


In [22]:
question_1 = "什么是南瓜书？"
question_2 = "Prompt Engineering for Developer是谁写的？"

In [23]:
result = qa_chain.invoke(question_1)
print("大模型+知识库后回答 question_1 的结果：")
print(result)

大模型+知识库后回答 question_1 的结果：
南瓜书是对《深度学习》这本书的昵称，由Ian Goodfellow、Yoshua Bengio和Aaron Courville合著。由于书的封面是橙色的，形似南瓜，因此得名。该书是深度学习领域的重要教材。谢谢你的提问！


In [24]:
result = qa_chain.invoke(question_2)
print("大模型+知识库后回答 question_2 的结果：")
print(result)

大模型+知识库后回答 question_2 的结果：
根据提供的上下文，我不知道《Prompt Engineering for Developer》这本书是谁写的。谢谢你的提问！


In [25]:
llm.invoke(question_1).content

'"南瓜书"是对《深度学习：算法与实现》的昵称，这本书由李沐、阿斯顿张、扎卡里·C·立顿和亚历山大·J·斯莫拉编写。之所以被称为"南瓜书"，是因为书的封面上有一个显眼的南瓜图案。这本书以其通俗易懂的语言和丰富的代码示例，广受深度学习初学者和实践者的欢迎。书中涵盖了深度学习的基本概念、常用算法以及如何使用深度学习框架进行实现。'

In [26]:
llm.invoke(question_2).content

'《Prompt Engineering for Developers》是由Isa Fulford和Andrew M. White撰写的。这本书旨在帮助开发者更好地理解和应用提示工程技术，以提高与大型语言模型的交互效果。'

#### 向向量添加聊天记录

In [27]:
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "你是一个问答任务的助手。 "
    "请使用检索到的上下文片段回答这个问题。 "
    "如果你不知道答案就说不知道。 "
    "请使用简洁的话语回答用户。"
    "\n\n"
    "{context}"
)

qa_prompt = ChatPromptTemplate([('system', system_prompt), ('placeholder', '{chat_history}'), ('human', '{input}')])

In [30]:
messages = qa_prompt.invoke({
  'input': '南瓜书是什么？',
  'chat_history': [],
  'context': ''
})

for message in messages.messages:
    print(message.content)

你是一个问答任务的助手。 请使用检索到的上下文片段回答这个问题。 如果你不知道答案就说不知道。 请使用简洁的话语回答用户。


南瓜书是什么？


In [31]:
messages = qa_prompt.invoke(
    {
        "input": "你可以介绍一下他吗？",
        "chat_history": [
            ("human", "西瓜书是什么？"),
            ("ai", "西瓜书是指周志华老师的《机器学习》一书，是机器学习领域的经典入门教材之一。"),
        ],
        "context": ""
    }
)
for message in messages.messages:
    print(message.content)

你是一个问答任务的助手。 请使用检索到的上下文片段回答这个问题。 如果你不知道答案就说不知道。 请使用简洁的话语回答用户。


西瓜书是什么？
西瓜书是指周志华老师的《机器学习》一书，是机器学习领域的经典入门教材之一。
你可以介绍一下他吗？


#### 带有信息压缩的检索链

In [32]:
from langchain_core.runnables import RunnableBranch

condense_question_system_template = (
    "请根据聊天记录完善用户最新的问题，"
    "如果用户最新的问题不需要完善则返回用户的问题。"
    )

condense_question_prompt = ChatPromptTemplate([
  ('system', condense_question_system_template),
  ('placeholder', '{chat_history}'),
  ('human', '{input}')
])

retrieve_docs = RunnableBranch(
    # 分支 1: 若聊天记录中没有 chat_history 则直接使用用户问题查询向量数据库
    (lambda x: not x.get("chat_history", False), (lambda x: x["input"]) | retriever, ),
    # 分支 2 : 若聊天记录中有 chat_history 则先让 llm 根据聊天记录完善问题再查询向量数据库
    condense_question_prompt | llm | StrOutputParser() | retriever,
)

In [33]:
# 重新定义 combine_docs
def combine_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs["context"]) # 将 docs 改为 docs["context"]
# 定义问答链
qa_chain = (
    RunnablePassthrough.assign(context=combine_docs) # 使用 combine_docs 函数整合 qa_prompt 中的 context
    | qa_prompt # 问答模板
    | llm
    | StrOutputParser() # 规定输出的格式为 str
)
# 定义带有历史记录的问答链
qa_history_chain = RunnablePassthrough.assign(
    context = (lambda x: x) | retrieve_docs # 将查询结果存为 content
    ).assign(answer=qa_chain) # 将最终结果存为 answer


In [34]:
# 不带聊天记录
qa_history_chain.invoke({
    "input": "西瓜书是什么？",
    "chat_history": []
})

{'input': '西瓜书是什么？',
 'chat_history': [],
 'context': [],
 'answer': '"西瓜书"是对《机器学习》一书的昵称，由周志华教授编写。由于书的封面是绿色的，并且有一个西瓜的图案，因此被称为"西瓜书"。这本书是机器学习领域的一本重要教材，广泛用于教学和自学。'}

In [35]:
# 带聊天记录
qa_history_chain.invoke({
    "input": "南瓜书跟它有什么关系？",
    "chat_history": [
        ("human", "西瓜书是什么？"),
        ("ai", "西瓜书是指周志华老师的《机器学习》一书，是机器学习领域的经典入门教材之一。"),
    ]
})


{'input': '南瓜书跟它有什么关系？',
 'chat_history': [('human', '西瓜书是什么？'),
  ('ai', '西瓜书是指周志华老师的《机器学习》一书，是机器学习领域的经典入门教材之一。')],
 'context': [],
 'answer': '南瓜书是指李航老师的《统计学习方法》一书。与西瓜书一样，南瓜书也是机器学习领域的重要教材，但侧重于统计学习方法。两者都是学习机器学习的常用参考书籍。'}

### 部署知识库助手

In [37]:
import streamlit as st
from langchain_openai import ChatOpenAI
import os
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableBranch, RunnablePassthrough
import sys
sys.path.append("./llm-universe/notebook/C3 搭建知识库") # 将父目录放入系统路径中
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma


In [38]:
def get_retriever():
    # 定义 Embeddings
    embedding = OpenAIEmbeddings()
    # 向量数据库持久化路径
    persist_directory = './llm-universe/data_base/vector_db/chroma'
    # 加载数据库
    vectordb = Chroma(
        persist_directory=persist_directory,
        embedding_function=embedding
    )
    return vectordb.as_retriever()


In [39]:
def combine_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs["context"])

In [40]:
def get_qa_history_chain():
    retriever = get_retriever()
    llm = ChatOpenAI(model_name="gpt-4o", temperature=0)
    condense_question_system_template = (
        "请根据聊天记录总结用户最近的问题，"
        "如果没有多余的聊天记录则返回用户的问题。"
    )
    condense_question_prompt = ChatPromptTemplate([
            ("system", condense_question_system_template),
            ("placeholder", "{chat_history}"),
            ("human", "{input}"),
        ])

    retrieve_docs = RunnableBranch(
        (lambda x: not x.get("chat_history", False), (lambda x: x["input"]) | retriever, ),
        condense_question_prompt | llm | StrOutputParser() | retriever,
    )

    system_prompt = (
        "你是一个问答任务的助手。 "
        "请使用检索到的上下文片段回答这个问题。 "
        "如果你不知道答案就说不知道。 "
        "请使用简洁的话语回答用户。"
        "\n\n"
        "{context}"
    )
    qa_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            ("placeholder", "{chat_history}"),
            ("human", "{input}"),
        ]
    )
    qa_chain = (
        RunnablePassthrough().assign(context=combine_docs)
        | qa_prompt
        | llm
        | StrOutputParser()
    )

    qa_history_chain = RunnablePassthrough().assign(
        context = retrieve_docs, 
        ).assign(answer=qa_chain)
    return qa_history_chain


In [41]:
def gen_response(chain, input, chat_history):
    response = chain.stream({
        "input": input,
        "chat_history": chat_history
    })
    for res in response:
        if "answer" in res.keys():
            yield res["answer"]


In [42]:
def main():
    st.markdown('### 🦜🔗 动手学大模型应用开发')
    # st.session_state可以存储用户与应用交互期间的状态与数据
    # 存储对话历史
    if "messages" not in st.session_state:
        st.session_state.messages = []
    # 存储检索问答链
    if "qa_history_chain" not in st.session_state:
        st.session_state.qa_history_chain = get_qa_history_chain()
    # 建立容器 高度为500 px
    messages = st.container(height=550)
    # 显示整个对话历史
    for message in st.session_state.messages: # 遍历对话历史
            with messages.chat_message(message[0]): # messages指在容器下显示，chat_message显示用户及ai头像
                st.write(message[1]) # 打印内容
    if prompt := st.chat_input("Say something"):
        # 将用户输入添加到对话历史中
        st.session_state.messages.append(("human", prompt))
        # 显示当前用户输入
        with messages.chat_message("human"):
            st.write(prompt)
        # 生成回复
        answer = gen_response(
            chain=st.session_state.qa_history_chain,
            input=prompt,
            chat_history=st.session_state.messages
        )
        # 流式输出
        with messages.chat_message("ai"):
            output = st.write_stream(answer)
        # 将输出存入st.session_state.messages
        st.session_state.messages.append(("ai", output))
